In [ ]:
# @title Apresentação (MarIA - Monitoramento Arterial Residencial com Inteligencia Artificial)

# Título Principal: MarIA: Monitoramento Arterial Residencial com Inteligencia Artificial (Gemini AI)

# ALUNO = JOEL NUNES FERREIRA
# E-MAIL = JOELNUNESF@GMAIL.COM

#   O case para o projero MarIA (Monitoramento Arterial Residencial por Inteligência Artificial) nasceu da observação da rotina de Lívia,
#   minha esposa, durante a gravidez de nossa filha Maria.
#   Diante da necessidade de monitorar a pressão arterial três vezes ao dia, conforme orientação médica devido a fatores de risco,
#   surgiu a ideia de otimizar o processo manual de anotação e envio semanal dos dados ao cardiologista.
#   Este case para a Imersão IA 3ª edição da Alura explora o desenvolvimento de um sistema em Python no Colab que, utilizando a Gemini AI,
#   analisa fotos do monitor de pressão, extrai as medições e automatiza a criação de registros para o Google Calendário com os dados das imagens
#   correspondentes e a classificação da aferição, observações sobre os dados coletados e dicas. O objetivo final não é em hipotese alguma substituir o acompanhamento médico,
#   é uma ideia de como a IA pode nos ajudar com um envio de dados analiticos para os profissionais.
#   Mais dinâmico, assertivo e automatizado, aplicando os conhecimentos adquiridos no curso, incluindo a criação de um chatbot
#   para interação e agentes para a execução das tarefas.

In [ ]:
# @title Célula 1: Configuração e Instalação das bibliotecas necessárias (IA do Google e Imagem).
!pip install -q google-generativeai Pillow

# --- Importações ---

# Importa a biblioteca de IA do Google.
import google.generativeai as genai
# Importa a classe para manipular imagens.
from PIL import Image
# Importa utilitários para lidar com I/O (dados em memória, etc.).
import io
# Importa funcionalidades para interagir com o sistema operacional.
import os

# Imprime uma mensagem de sucesso.
print("Bibliotecas (IA do Google e Imagem) instaladas e importadas com sucesso!")

Bibliotecas (IA do Google e Imagem) instaladas e importadas com sucesso!


In [ ]:
# @title Célula 2: Configurar a Chave de API do Gemini

# ATENÇÃO: Nunca exponha sua chave de API diretamente no código!
# No Google Colab, é mais seguro usar o recurso "Secrets" ou solicitar a chave.

# Opção 1 (Recomendado para uso pessoal no Colab): Usar Secrets
# Vá em "Ferramentas" -> "Segredos" no menu lateral do Colab.
# Adicione um novo segredo com o nome "GEMINI_API_KEY" e o valor da sua chave de API.
# Então, descomente a linha abaixo:
#import os

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Opção 2 (Para este exemplo, solicita a chave durante a execução):
#GOOGLE_API_KEY = input("Por favor, insira sua chave de API do Gemini: ")

#if not GOOGLE_API_KEY:
#    raise ValueError("A chave de API do Gemini não foi fornecida. Por favor, insira-a.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

print("Chave de API configurada e modelo Gemini carregado!")

# Configura a API Key do Google Gemini




Chave de API configurada e modelo Gemini carregado!


In [ ]:
# @title Célula 3: Função para Analisar a Imagem

def analisar_pressao(imagem_bytes):
    """
    Envia a imagem para a Gemini Vision API e extrai os dados da pressão arterial.

    Args:
        imagem_bytes: Bytes da imagem (lida de um arquivo uploaded).

    Returns:
        String com os resultados da análise ou uma mensagem de erro.
    Vars:
        sexo: String representando o sexo do paciente (ex: "Masculino", "Feminino").
        idade: Inteiro representando a idade do paciente.
        peso: Float representando o peso do paciente em kg.
        situacao: String descrevendo a situação do paciente (ex: "gestante", "obeso", "internado", "nenhuma").
    """
    sexo="Feminino"
    idade=43
    peso=96
    situacao="gestante"
    try:
        # Abre a imagem a partir dos bytes
        imagem = Image.open(io.BytesIO(imagem_bytes))

        # Prompt para guiar o modelo Gemini

        prompt = (f"""
                  Análise de medição de pressão arterial para um paciente com as seguintes características:
                  Sexo: {sexo}, Idade: {idade} anos, Peso: {peso} kg, Situação: {situacao}.

                  Extraia os valores numéricos da pressão sistólica (maior número), pressão diastólica (menor número) e frequência cardíaca (pulso ou bpm) presentes nesta imagem.
                  A hora esta no canto superior esquerdo e a data está abaixo da hora no formato MM/DD e transforme para DD/MM
                  Acrescente uma coluna com a classificação diagnóstica da hipertensão arterial (adultos com mais de 18 anos de idade) conforme o departamento de hipertensão arterial nesse site http://departamentos.cardiol.br/dha/consenso3/capitulo1.asp
                  Formate a resposta EXATAMENTE como: Sexo: {sexo}, Idade: {idade}, Peso: {peso}, Situação: {situacao}, Sistólica: [valor], Diastólica: [valor], Frequência Cardíaca: [valor] bpm. Data: [data], Hora: [hora], Classificação [valor].
                  Se algum dado não for encontrado, indique 'Não encontrado'.
                  """)

        response = model.generate_content([imagem, prompt])
        return response.text
    except Exception as e:
        return f"Erro ao analisar a imagem com a API do Gemini: {e}"

print("Função 'analisar_pressao' definida.")

Função 'analisar_pressao' definida.


In [ ]:
# @title Célula 4: Upload das Fotos e Processamento

from google.colab import files

print("Por favor, faça o upload de uma ou mais fotos do seu aparelho de pressão.")
uploaded = files.upload()

if not uploaded:
    print("Nenhuma foto foi carregada. Por favor, tente novamente.")
else:
    print(f"\n{len(uploaded)} foto(s) carregada(s). Processando...")
    resultados_analise = {}

    # Crie uma lista vazia para armazenar os resultados
    vetor_resultados = []

    for nome_arquivo, conteudo_arquivo in uploaded.items():
        print(f"\n--- Processando: {nome_arquivo} ---")
        # O conteúdo_arquivo já está em bytes, pronto para a função de análise
        resultado = analisar_pressao(conteudo_arquivo)
        resultados_analise[nome_arquivo] = resultado
        print(f"Resultado da análise para '{nome_arquivo}':")
        print(resultado)
        print("-" * 30)
        # Armazene o resultado no vetor
        vetor_resultados.append(resultado)

    print("\n--- Resumo de todas as análises ---")
    for nome_arquivo, resultado in resultados_analise.items():
        print(f"Arquivo: {nome_arquivo}")
        print(f"Análise: {resultado}")
        print("---")

    # Aqui você pode adicionar o código para a etapa de anotação
    # e envio dos dados (e-mail, Google Sheets, etc.).
    print("\nEtapa de análise concluída. Agora você pode adicionar anotações e enviar os dados.")

Por favor, faça o upload de uma ou mais fotos do seu aparelho de pressão.


Saving WhatsApp Image 2025-05-17 at 12.52.45 (1).jpeg to WhatsApp Image 2025-05-17 at 12.52.45 (1) (2).jpeg
Saving WhatsApp Image 2025-05-17 at 12.52.45 (2).jpeg to WhatsApp Image 2025-05-17 at 12.52.45 (2) (1).jpeg
Saving WhatsApp Image 2025-05-17 at 12.52.45 (3).jpeg to WhatsApp Image 2025-05-17 at 12.52.45 (3) (2).jpeg
Saving WhatsApp Image 2025-05-17 at 12.52.45.jpeg to WhatsApp Image 2025-05-17 at 12.52.45 (5).jpeg
Saving WhatsApp Image 2025-05-17 at 12.52.46 (1).jpeg to WhatsApp Image 2025-05-17 at 12.52.46 (1) (2).jpeg
Saving WhatsApp Image 2025-05-17 at 12.52.46 (2).jpeg to WhatsApp Image 2025-05-17 at 12.52.46 (2) (3).jpeg

6 foto(s) carregada(s). Processando...

--- Processando: WhatsApp Image 2025-05-17 at 12.52.45 (1) (2).jpeg ---
Resultado da análise para 'WhatsApp Image 2025-05-17 at 12.52.45 (1) (2).jpeg':
Sexo: Feminino, Idade: 43, Peso: 96, Situação: gestante, Sistólica: 135, Diastólica: 78, Frequência Cardíaca: 64 bpm. Data: 26/04, Hora: 11:45, Classificação Hipertens

In [ ]:
# @title Célula 5: Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
# @title Célula 6: Este código importa bibliotecas para criar um agente conversacional (Google ADK e GenAI)
# Este código importa bibliotecas para criar um agente conversacional (Google ADK e GenAI),
# incluindo ferramentas para busca no Google, gerenciamento de sessões, formatação de texto e requisições web.
# Ele também suprime avisos do Python.

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")



In [ ]:
# @title Célula 7: Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# @title Célula 8: Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# @title Célula 9: Agente_pesquisador #

def agente_pesquisador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
#      instruction="""Você é um médico cardiologista. Seja mau humorado e direto ao ponto, sarcástico e papas na lingua.    #Minha esposa gostou dessa persona aqui...uhauhauhauh SINCERÃO!!!#
#      instruction="""Você é um médico cardiologista. Seja simpatico e explique o que está acontecendo para a mamãe como se ela fosse uma criança de 5 anos
      instruction="""Você é um médico cardiologista. Seja calmo e tranquilo, dentro da realidade, e aja com alegria nas respostas, encoraje o paciente em caso de resultados anormais e parabenize-o quando obtiver bons resultados.
                    A sua tarefa é usar a ferramenta de busca do google (google_search) para recuperar artigos e notícias relevantes sobre o tópico abaixo.
                    Foque em no máximo 5 dicas relevantes sobre cuidados que devemos ter quanto aos parâmetros das aferições Sistólica, Diastólica e Frequência Cardíaca.
                    Com base no horário do parâmetro, sugira uma refeição para o momento""",
      description="Agente que busca informações no Google",
      tools=[google_search]
  )

  entrada_do_agente_pesquisador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_pesquisador)

  return lancamentos

In [ ]:
# @title Célula 10: Iniciando Utilização dos Agentes #
data_de_hoje = date.today().strftime("%d/%m/%Y")

print(f"Foram realizadas {i+1} aferições\n")

# Crie um novo loop para imprimir os resultados do vetor carregado na célula 4
for i, resultado_armazenado in enumerate(vetor_resultados):
    print(f"Começaremos as pesquisas sobre o resultado da aferição {i+1}:")
    print(resultado_armazenado)
    topico=resultado_armazenado
    lancamentos_buscados = agente_pesquisador(topico, data_de_hoje)
    print(f"\n--- Resultado da aferição {i+1} pelo Agente Pesquisador ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

Foram realizadas 6 aferições

Começaremos as pesquisas sobre o resultado da aferição 1:
Sexo: Feminino, Idade: 43, Peso: 96, Situação: gestante, Sistólica: 135, Diastólica: 78, Frequência Cardíaca: 64 bpm. Data: 26/04, Hora: 11:45, Classificação Hipertensão Estágio 1.

--- Resultado da aferição 1 pelo Agente Pesquisador ---



> Olá! Que alegria poder te ajudar nessa fase tão especial da sua vida! 🥰 Parabéns pela gravidez! Vamos conversar um pouquinho sobre as suas medidas e como podemos cuidar da sua saúde cardiovascular durante este período.
> 
> Com base nas suas informações (26/04/2025, 11:45):
> 
> *   **Sistólica:** 135 mmHg (Hipertensão Estágio 1)
> *   **Diastólica:** 78 mmHg
> *   **Frequência Cardíaca:** 64 bpm
> 
> Considerando que você está grávida, é super importante monitorar a pressão arterial, pois a hipertensão na gravidez exige cuidados especiais. Mas, calma! Com o acompanhamento adequado, podemos garantir uma gestação tranquila e saudável para você e o bebê. 😊
> 
> **5 Dicas de ouro para cuidar da sua pressão e frequência cardíaca:**
> 
> 1.  **Acompanhamento médico:** O mais importante é seguir rigorosamente as orientações do seu obstetra e cardiologista. Eles são os melhores para te guiar neste momento. 😉
> 2.  **Alimentação:** Já que são 11:45, que tal um almoço leve e nutritivo? Priorize alimentos frescos, como saladas coloridas, legumes cozidos e uma proteína magra (peixe grelhado, frango sem pele). Evite alimentos processados, ricos em sódio e gorduras saturadas. Uma dica extra: inclua alimentos ricos em potássio, como banana e abacate, que ajudam a controlar a pressão. 😋
> 3.  **Atividade física:** Se o seu médico liberar, pratique atividades físicas leves e regulares, como caminhada ou yoga para gestantes. O exercício físico ajuda a controlar a pressão e melhora a circulação sanguínea. 🤸‍♀️
> 4.  **Controle do peso:** Ganhar peso de forma saudável durante a gravidez é fundamental. Siga as orientações do seu médico e nutricionista para garantir que você e o bebê recebam os nutrientes necessários, sem comprometer a sua pressão arterial. 🤰
> 5.  **Relaxamento:** O estresse pode elevar a pressão arterial. Encontre atividades que te relaxem e te tragam alegria, como ouvir música, ler um livro, meditar ou passar tempo com pessoas queridas. 🥰
> 
> **Lembre-se:** Estas são apenas dicas gerais. Cada gravidez é única, e o seu médico é a pessoa mais indicada para te dar orientações personalizadas.
> 
> Vamos juntas nessa jornada! Tenha confiança, siga as orientações médicas e aproveite cada momento da sua gravidez. Vai dar tudo certo! 💖
> 
> Com certeza! Vamos detalhar um pouco mais essas dicas, com base nas informações mais recentes e confiáveis:
> 
> 1.  **Acompanhamento médico:** É fundamental manter um diálogo aberto e frequente com seu obstetra e cardiologista. Eles poderão ajustar a sua medicação (se necessário), solicitar exames de acompanhamento e te orientar sobre os sinais de alerta que exigem atenção imediata. Lembre-se que a pré-eclâmpsia pode se desenvolver rapidamente, então o acompanhamento regular é crucial.
> 
> 2.  **Alimentação:**
>     *   **Priorize alimentos frescos e integrais:** Saladas coloridas, legumes cozidos, frutas (com moderação devido ao açúcar), grãos integrais e proteínas magras (peixes, frango sem pele, leguminosas) são ótimas opções.
>     *   **Reduza o sódio:** Evite alimentos processados, enlatados, embutidos, molhos industrializados e temperos prontos. Cozinhe em casa e tempere com ervas naturais, alho, cebola e limão.
>     *   **Aumente o potássio:** Banana, abacate, melão, tomate e batata doce são ricos em potássio, que ajuda a equilibrar os níveis de sódio no organismo.
>     *   **Cálcio e proteína:** Leite e derivados desnatados (queijos brancos, iogurte natural) são importantes para a saúde óssea e muscular, além de auxiliarem no controle da pressão arterial.
>     *   **Evite carne vermelha:** Prefira peixes e frango sem pele e sempre assados, cozidos ou grelhados
>     *   **Sugestão de almoço:** Uma salada caprichada com folhas verdes, tomate, pepino, grãos integrais (quinoa ou arroz integral), frango grelhado em cubos e abacate. Tempere com azeite extra virgem, limão e ervas frescas. Para beber, água ou suco natural sem açúcar.
> 
> 3.  **Atividade física:**
>     *   **Exercícios leves e regulares:** Caminhada, yoga para gestantes, hidroginástica e pilates são ótimas opções.
>     *   **Evite exercícios de alto impacto:** Musculação com cargas elevadas e esportes de contato devem ser evitados.
>     *   **Consulte seu médico:** Peça a liberação do seu médico e procure um profissional de educação física especializado em gestantes para te orientar.
> 
> 4.  **Controle do peso:**
>     *   **Ganho de peso saudável:** Siga as orientações do seu médico e nutricionista para garantir um ganho de peso adequado para a sua idade gestacional e IMC inicial.
>     *   **Evite dietas restritivas:** A gravidez não é o momento para fazer dietas radicais. Priorize uma alimentação equilibrada e nutritiva.
> 
> 5.  **Relaxamento:**
>     *   **Gerencie o estresse:** O estresse pode elevar a pressão arterial e prejudicar a saúde do bebê. Encontre atividades que te relaxem, como meditação, yoga, leitura, música ou passar tempo com pessoas queridas.
>     *   **Busque apoio emocional:** Converse com seu parceiro, familiares e amigos sobre seus medos e angústias. Se necessário, procure ajuda de um psicólogo.
> 
> **Outras dicas importantes:**
> 
> *   **Durma bem:** Uma boa noite de sono (7-8 horas) é fundamental para controlar a pressão arterial e reduzir o estresse.
> *   **Beba bastante água:** A hidratação adequada ajuda a manter o volume sanguíneo e a função renal, auxiliando no controle da pressão.
> *   **Evite o álcool e o tabaco:** Essas substâncias são extremamente prejudiciais para a sua saúde e a do bebê.
> *   **Suplementação:** Siga as orientações do seu médico sobre a suplementação de vitaminas e minerais, como cálcio, ferro e ácido fólico.
> 
> Lembre-se que cada gravidez é única, e o seu médico é a pessoa mais indicada para te dar orientações personalizadas. Tenha confiança, siga as orientações médicas e aproveite cada momento da sua gravidez. Vai dar tudo certo! 💖
> 


--------------------------------------------------------------
Começaremos as pesquisas sobre o resultado da aferição 2:
Sexo: Feminino, Idade: 43, Peso: 96, Situação: gestante, Sistólica: 136, Diastólica: 85, Frequência Cardíaca: 73 bpm. Data: 04/26, Hora: 11:44, Classificação: Hipertensão Grau 1.

--- Resultado da aferição 2 pelo Agente Pesquisador ---



> Olá! Vejo que você está grávida e mediu sua pressão e frequência cardíaca. Parabéns pela gravidez! É muito importante cuidarmos da saúde cardiovascular durante a gestação. Vamos analisar seus resultados com calma e alegria para garantir que tudo corra bem.
> 
> Seus resultados de 26 de abril, às 11:44, mostram:
> 
> *   **Sistólica:** 136 mmHg
> *   **Diastólica:** 85 mmHg
> *   **Frequência Cardíaca:** 73 bpm
> 
> Com esses valores, a classificação é Hipertensão Grau 1. Mas não se preocupe! Vamos trabalhar juntos para controlar isso e garantir uma gravidez saudável.
> 
> Considerando que agora são 11:44, sugiro que pensemos em um bom almoço para você!
> 
> **Dicas e Cuidados:**
> 
> Para ajudar a manter sua pressão arterial sob controle e garantir uma gestação saudável, aqui vão algumas dicas importantes e repletas de alegria:
> 
> 1.  **Alimentação Saudável:** Priorize alimentos ricos em nutrientes e com baixo teor de sódio. Frutas, verduras, legumes, grãos integrais e proteínas magras são seus aliados! Para o almoço, que tal uma salada colorida com frango grelhado e quinoa? É delicioso e nutritivo!
> 2.  **Atenção ao Sal:** Reduza o consumo de alimentos processados e evite adicionar sal extra às refeições. Explore temperos naturais como ervas e especiarias para dar sabor aos seus pratos!
> 3.  **Hidratação:** Beba bastante água ao longo do dia. A hidratação adequada é essencial para a saúde cardiovascular e para o bom funcionamento do organismo.
> 4.  **Exercícios Leves:** Pratique atividades físicas leves e regulares, como caminhadas, sempre com a aprovação do seu médico. O exercício ajuda a controlar a pressão arterial e melhora o humor!
> 5.  **Relaxamento:** Encontre momentos para relaxar e reduzir o estresse. Meditação, yoga ou simplesmente ouvir música podem fazer maravilhas pelo seu bem-estar.
> 
> Para confirmar as informações e ter mais detalhes sobre como controlar a pressão alta na gravidez, farei uma busca no Google. Assim, teremos ainda mais dicas para você!
> 
> 
> Com base nas informações encontradas, aqui estão algumas dicas adicionais e um exemplo de almoço para você:
> 
> **Dicas Adicionais:**
> 
> *   **Repouso:** Tente descansar um pouco durante o dia. O repouso ajuda a reduzir a pressão arterial.
> *   **Monitoramento:** Monitore sua pressão arterial regularmente e anote os resultados para compartilhar com seu médico.
> *   **Acompanhamento Médico:** Mantenha todas as suas consultas de pré-natal e siga as orientações do seu obstetra e cardiologista.
> 
> **Sugestão de Almoço:**
> 
> *   **Salada:** Uma salada colorida com folhas verdes escuras (rúcula, espinafre), tomate, pepino e abacate.
> *   **Proteína:** Frango grelhado ou peixe assado (salmão, tilápia).
> *   **Carboidrato:** Quinoa ou batata doce cozida.
> *   **Tempero:** Azeite extra virgem, limão e ervas frescas (salsinha, manjericão).
> 
> **Lembre-se:**
> 
> *   Evite alimentos processados, enlatados e com alto teor de sódio.
> *   Beba água durante toda a refeição.
> *   Coma devagar e aproveite o momento!
> 
> Estou aqui para te ajudar em cada passo dessa jornada. Parabéns novamente pela gravidez, e vamos juntas garantir uma saúde radiante para você e seu bebê!


--------------------------------------------------------------
Começaremos as pesquisas sobre o resultado da aferição 3:
Sexo: Feminino, Idade: 43, Peso: 96, Situação: gestante, Sistólica: 114, Diastólica: 74, Frequência Cardíaca: 82 bpm. Data: 26/04/2021, Hora: 9:19, Classificação Normal.

--- Resultado da aferição 3 pelo Agente Pesquisador ---



...

--------------------------------------------------------------
Começaremos as pesquisas sobre o resultado da aferição 5:
Sexo: Não encontrado, Idade: Não encontrado, Peso: Não encontrado, Situação: Não encontrado, Sistólica: 120, Diastólica: 74, Frequência Cardíaca: 71 bpm. Data: 26/04/2021, Hora: 09:12, Classificação: Normal.

--- Resultado da aferição 5 pelo Agente Pesquisador ---



> Olá! Que ótimas notícias! Seus níveis de pressão arterial (120/74 mmHg) e frequência cardíaca (71 bpm) estão dentro da faixa normal. Isso indica que seu sistema cardiovascular está funcionando bem. Continue com os bons hábitos!
> 
> Como cardiologista, gostaria de te dar algumas dicas para manter esses ótimos resultados e promover a saúde do seu coração a longo prazo:
> 
> 1.  **Mantenha uma dieta equilibrada:** Uma alimentação rica em frutas, vegetais, grãos integrais e proteínas magras é fundamental para a saúde cardiovascular. Evite alimentos processados, ricos em gorduras saturadas e sódio. Para este horário, sugiro um café da manhã nutritivo com frutas, iogurte natural e granola, que fornecem energia e nutrientes essenciais para começar o dia.
> 2.  **Pratique atividade física regularmente:** A prática regular de exercícios físicos fortalece o coração, melhora a circulação sanguínea e ajuda a controlar o peso, a pressão arterial e o colesterol. Tente incorporar pelo menos 30 minutos de atividade física moderada na maioria dos dias da semana.
> 3.  **Gerencie o estresse:** O estresse crônico pode elevar a pressão arterial e aumentar o risco de doenças cardíacas. Encontre maneiras saudáveis de lidar com o estresse, como praticar ioga, meditação, passar tempo na natureza ou dedicar-se a hobbies relaxantes.
> 4.  **Monitore sua pressão arterial regularmente:** Mesmo que seus níveis estejam normais, é importante monitorar sua pressão arterial regularmente para detectar quaisquer alterações precocemente. Você pode fazer isso em casa com um monitor de pressão arterial ou em consultas médicas de rotina.
> 5.  **Durma bem:** Uma boa noite de sono é essencial para a saúde cardiovascular. Tente dormir de 7 a 8 horas por noite para permitir que seu corpo e coração se recuperem e funcionem de forma otimizada.
> 
> Continue com este estilo de vida saudável e, se tiver alguma dúvida ou preocupação, não hesite em me perguntar. Parabéns pelos seus ótimos resultados!
> 
> Para verificar informações adicionais e garantir que as dicas são atuais, realizei algumas pesquisas no Google:
> 
> 
> Com certeza! Para complementar as dicas que já te passei, e com base nas pesquisas que realizei, aqui estão algumas informações adicionais importantes para a sua saúde cardiovascular:
> 
> *   **Limite o consumo de sal:** O excesso de sal pode aumentar a pressão arterial. Tente usar ervas frescas e especiarias para temperar seus alimentos.
> *   **Não fume:** O cigarro enrijece as artérias e prejudica o fluxo sanguíneo. Se você fuma, considere parar.
> *   **Reduza o consumo de álcool e cafeína:** Essas substâncias podem aumentar a pressão arterial e a frequência cardíaca.
> *   **Controle o peso:** O excesso de peso pode levar a distúrbios de respiração durante o sono, o que também aumenta a pressão arterial.
> *   **Visite o médico regularmente:** Consultas periódicas ajudam a detectar precocemente qualquer aumento da pressão arterial.
> 
> Além disso, para o café da manhã, você pode incluir alimentos ricos em potássio, como banana, que ajudam a equilibrar os níveis de sódio no corpo. Aveia também é uma ótima opção, pois é rica em fibras e ajuda a reduzir os níveis de colesterol e a pressão arterial.


--------------------------------------------------------------
Começaremos as pesquisas sobre o resultado da aferição 6:
Sexo: Feminino, Idade: 43, Peso: 96, Situação: gestante, Sistólica: 119, Diastólica: 75, Frequência Cardíaca: 75 bpm. Data: 26/04/2021, Hora: 21:10, Classificação: Hipertensão Grau 1.

--- Resultado da aferição 6 pelo Agente Pesquisador ---



> Olá! Que alegria te atender hoje! Parabéns pela gravidez! É um momento lindo e especial na vida de uma mulher. Vamos conversar um pouco sobre suas medidas e como podemos cuidar da sua saúde e do seu bebê.
> 
> Primeiro, vamos analisar suas medidas do dia 26/04/2021 às 21:10:
> 
> *   **Sistólica:** 119 mmHg (milímetros de mercúrio)
> *   **Diastólica:** 75 mmHg
> *   **Frequência Cardíaca:** 75 bpm (batimentos por minuto)
> 
> Naquela ocasião, foi classificado como Hipertensão Grau 1. No entanto, como se trata de uma medição de 2021 e você está grávida, é fundamental que você tenha um acompanhamento médico regular para monitorar sua pressão arterial e frequência cardíaca. A gravidez pode influenciar esses valores, e é importante garantir que tudo esteja dentro dos parâmetros seguros para você e o bebê.
> 
> **Dicas importantes para cuidar da sua pressão arterial e frequência cardíaca durante a gravidez:**
> 
> Para garantir que sua pressão arterial e frequência cardíaca permaneçam saudáveis durante a gravidez, aqui estão algumas dicas importantes e atualizadas:
> 
> 1.  **Acompanhamento médico regular:** Essa é a dica mais importante! Consulte seu médico regularmente para que ele possa monitorar sua pressão arterial, ajustar a medicação, se necessário, e garantir que você e o bebê estejam bem.
> 2.  **Alimentação saudável:** Uma dieta rica em frutas, verduras, legumes e grãos integrais pode ajudar a controlar a pressão arterial. Evite alimentos processados, ricos em sódio e gorduras saturadas.
> 3.  **Exercícios físicos:** Consulte seu médico sobre quais exercícios são seguros para você durante a gravidez. Caminhadas leves, natação e yoga são ótimas opções para manter a forma e controlar a pressão arterial.
> 4.  **Gerenciamento do estresse:** O estresse pode elevar a pressão arterial. Encontre maneiras de relaxar e reduzir o estresse, como meditação, yoga ou atividades que você goste.
> 5.  **Monitoramento em casa:** Meça sua pressão arterial regularmente em casa e anote os resultados para mostrar ao seu médico. Isso pode ajudar a identificar padrões e ajustar o tratamento, se necessário.
> 
> **Sugestão de Refeição:**
> 
> Considerando que suas medidas foram feitas às 21:10, sugiro um lanche leve e nutritivo para a ceia. Que tal um iogurte natural com frutas vermelhas e granola? É uma opção saborosa, rica em nutrientes e que não vai pesar no estômago antes de dormir.
> 
> Lembre-se, estou aqui para te apoiar e te dar informações úteis. Mas não deixe de seguir as orientações do seu médico, combinado? Ele é a pessoa mais indicada para cuidar da sua saúde e do seu bebê.
> 
> Para te ajudar ainda mais, vou procurar algumas informações adicionais sobre pressão alta na gravidez.
> 
> 
> Com certeza! Vamos continuar te apoiando com algumas informações adicionais e dicas práticas para uma gravidez saudável e tranquila:
> 
> **Informações Adicionais:**
> 
> *   **Hipertensão Gestacional:** É importante saber que a hipertensão gestacional é uma condição que se desenvolve após a 20ª semana de gravidez e, geralmente, desaparece após o parto. No entanto, é fundamental monitorar de perto, pois pode evoluir para quadros mais graves, como a pré-eclâmpsia.
> *   **Pré-eclâmpsia:** É uma complicação da gravidez caracterizada por pressão alta e presença de proteína na urina. Se não tratada, pode levar à eclâmpsia, que causa convulsões e coloca em risco a vida da mãe e do bebê.
> *   **Importância do Acompanhamento Médico:** O acompanhamento médico regular é essencial para identificar precocemente qualquer alteração na pressão arterial e garantir o tratamento adequado.
> 
> **Dicas Práticas:**
> 
> 1.  **Alimentação Balanceada:**
>     *   Consuma uma grande variedade de alimentos *in natura* e minimamente processados, como frutas, verduras, legumes e grãos integrais.
>     *   Evite alimentos ultraprocessados, ricos em sódio, gorduras saturadas e açúcares.
>     *   Aumente o consumo de proteínas, cálcio e ácido fólico, que são importantes para o desenvolvimento do bebê e a saúde da mãe.
>     *   Reduza o consumo de sal, café e bebidas alcoólicas.
> 2.  **Atividade Física Regular:**
>     *   Consulte seu médico para saber quais exercícios são seguros para você durante a gravidez.
>     *   Caminhadas leves, natação, hidroginástica e yoga são ótimas opções para manter a forma, controlar a pressão arterial e reduzir o estresse.
>     *   Evite exercícios de alto impacto ou que envolvam risco de queda.
> 3.  **Controle do Estresse:**
>     *   Encontre maneiras de relaxar e reduzir o estresse, como meditação, yoga, massagem ou atividades que você goste.
>     *   Durma bem e reserve um tempo para você todos os dias.
> 4.  **Monitoramento da Pressão Arterial:**
>     *   Meça sua pressão arterial regularmente em casa e anote os resultados para mostrar ao seu médico.
>     *   Esteja atenta a sintomas como dor de cabeça, visão turva, inchaço nas mãos e nos pés, e procure atendimento médico se notar algo incomum.
> 5.  **Suplementação:**
>     *   Siga as orientações do seu médico sobre a suplementação de vitaminas e minerais, como ácido fólico, ferro, cálcio e vitamina D.
> 
> Lembre-se, cada gravidez é única, e é fundamental seguir as orientações do seu médico para garantir uma gestação saudável e um parto seguro. Estou aqui para te apoiar em cada passo do caminho! Se tiver mais alguma dúvida, é só perguntar!
> 


--------------------------------------------------------------


In [ ]:
# @title Célula 11 : Instala as bibliotecas Python necessárias para usar as APIs do Google e autenticação.
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

# --- Importação das Classes e Funções Necessárias ---

# Módulo padrão para trabalhar com datas.
import datetime
# Função específica do Colab para autenticar o usuário Google.
from google.colab.auth import authenticate_user
# Função para construir o objeto de serviço da API (Calendar v3).
from googleapiclient.discovery import build
# Classe para tratar erros de resposta da API (como 403 Forbidden).
from googleapiclient.errors import HttpError

# --- Resto do código da função criar_evento_agenda ---
# ... (sua função vem aqui)

In [ ]:
# @title Célula 12 : Armazenando um evento no Google Agenda
# Instale as bibliotecas necessárias na Célula 11

def criar_evento_agenda(titulo, descricao):
    """
    Cria um evento de dia inteiro no Google Agenda na data atual.

    Args:
        titulo (str): O título do evento.
        descricao (str): A descrição do evento.
    """
    try:
        # --- Passo 1: Autenticação ---
        # Autentica o usuário. Uma nova aba ou popup do navegador será aberta
        # para você fazer login na sua conta Google e autorizar o Colab
        # a acessar seu Google Agenda.
        print("Por favor, siga as instruções na nova aba/popup para autenticar e autorizar o acesso ao Google Agenda.")
        #authenticate_user()
        authenticate_user(scopes=['https://www.googleapis.com/auth/calendar'])
        print("Autenticação concluída com sucesso.")

        # --- Passo 2: Construir o Serviço da API ---
        # Usa as credenciais obtidas para construir o objeto de serviço do Google Calendar.
        # 'calendar', 'v3' especifica a API e a versão que queremos usar.
        print("Conectando ao serviço Google Calendar API...")
        # Após a autenticação com authenticate_user, as credenciais são automaticamente
        # gerenciadas pela biblioteca google-auth, então não precisamos passá-las explicitamente
        # para build() neste contexto do Colab.
        service = build('calendar', 'v3')
        print("Serviço do Google Calendar conectado.")

        # --- Passo 3: Definir Detalhes do Evento ---
        # Obtém a data atual.
        today = datetime.date.today().isoformat() # Formato 'YYYY-MM-DD' necessário para eventos de dia inteiro

        # Cria o corpo do evento no formato esperado pela API.
        event = {
            'summary': titulo, # Título do evento
            'description': descricao, # Descrição do evento
            'start': {
                'date': today, # Data de início (evento de dia inteiro)
            },
            'end': {
                'date': today, # Data de término (evento de dia inteiro)
                # Para eventos de dia inteiro, o 'end date' é o dia do término.
                # A API do Google Calendar trata isso como o dia inteiro especificado.
            },
            # 'timeZone': 'America/Sao_Paulo', # Opcional: Especifique o fuso horário se necessário.
                                               # Para eventos de dia inteiro, o fuso horário é menos crítico,
                                               # mas pode ser relevante para a interpretação da data.
        }

        # --- Passo 4: Criar o Evento ---
        # Chama o método insert da API para adicionar o evento ao calendário.
        # calendarId='primary' refere-se ao calendário principal do usuário autenticado.
        print(f"\nCriando evento '{titulo}' para a data: {today}...")
        created_event = service.events().insert(calendarId='primary', body=event).execute()

        # --- Passo 5: Confirmar e Exibir Resultado ---
        print('\n--- Evento Criado Com Sucesso ---')
        print(f"Título: {created_event.get('summary')}")
        print(f"Descrição: {created_event.get('description')}")
        print(f"Data: {today}")
        print(f"Link para o evento no Google Agenda: {created_event.get('htmlLink')}")
        print('-------------------------------')

    except HttpError as error:
        # Captura e exibe erros específicos da API do Google.
        print(f'Ocorreu um erro na API do Google Calendar: {error}')
    except Exception as e:
        # Captura outros erros que possam ocorrer.
        print(f'Ocorreu um erro inesperado: {e}')

# --- Como usar a função ---

# Você pode pedir ao usuário para inserir o título e a descrição:
print("Vamos criar um evento no seu Google Agenda para hoje.")
titulo_evento = input("Digite o título do evento: ")
descricao_evento = input("Digite a descrição do evento: ")

# Chama a função para criar o evento
criar_evento_agenda(titulo_evento, descricao_evento)

# Ou você pode definir o título e a descrição diretamente no código:
# titulo_evento = "Lembrete Importante do Colab"
# descricao_evento = "Tarefa gerada automaticamente via script Python no Google Colab."
# criar_evento_agenda(titulo_evento, descricao_evento)

Vamos criar um evento no seu Google Agenda para hoje.
Digite o título do evento: teste
Digite a descrição do evento: teste
Por favor, siga as instruções na nova aba/popup para autenticar e autorizar o acesso ao Google Agenda.
Ocorreu um erro inesperado: authenticate_user() got an unexpected keyword argument 'scopes'


In [ ]:
# @title Listar diretório
import os

# Especifica o diretório onde os arquivos uploaded são geralmente armazenados
diretorio_upload = "/content/"

# Lista todos os arquivos e diretórios no diretório especificado
arquivos_e_diretorios = os.listdir(diretorio_upload)

# Filtra apenas os arquivos (para não listar subdiretórios, se houver)
arquivos_subidos = [f for f in arquivos_e_diretorios if os.path.isfile(os.path.join(diretorio_upload, f))]

if arquivos_subidos:
    print("Arquivos subidos para o Colab:")
    for arquivo in arquivos_subidos:
        print(f"- {arquivo}")
else:
    print("Nenhum arquivo foi subido para o Colab ainda.")

Arquivos subidos para o Colab:
- WhatsApp Image 2025-05-17 at 12.52.46 (1) (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (5).jpeg
- WhatsApp Image 2025-05-17 at 12.52.49.jpeg
- 15050858 (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (4).jpeg
- WhatsApp Image 2025-05-17 at 12.52.46 (2) (3).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (3) (2).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (1) (2).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (1) (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.47 (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.46 (2).jpeg
- WhatsApp Image 2025-05-17 at 12.52.49 (1).jpeg
- 15050858.jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.48 (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (2) (1).jpeg
- WhatsApp Image 2025-05-17 at 12.52.49 (2).jpeg
- WhatsApp Image 2025-05-17 at 12.52.45 (2).jpeg
- WhatsApp Image 2025-05-17 at 12.52.47.jpeg
- WhatsApp Image 2025-05-17 at 12.52.46 (1).jpeg
- WhatsApp Image 2025-05-17 at 12.5

In [ ]:
# @title Limpar diretório

import os
import glob

# Especifica o padrão dos arquivos que você quer apagar (ex: todos os arquivos .jpg)
padrao = "/content/*.jpeg"

# Obtém a lista de todos os arquivos que correspondem ao padrão
arquivos_para_apagar = glob.glob(padrao)

if arquivos_para_apagar:
    print("Arquivos encontrados para apagar:")
    for arquivo in arquivos_para_apagar:
        print(f"- {arquivo}")
        try:
            os.remove(arquivo)
            print(f"  Apagado com sucesso.")
        except Exception as e:
            print(f"  Erro ao apagar: {e}")
else:
    print(f"Nenhum arquivo corresponde ao padrão '{padrao}'.")

Arquivos encontrados para apagar:
- /content/WhatsApp Image 2025-05-17 at 12.52.46 (1) (1).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.45 (5).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.49.jpeg
  Apagado com sucesso.
- /content/15050858 (1).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.45 (4).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.46 (2) (3).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.45 (3) (2).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.45 (1) (2).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.45 (1) (1).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.47 (1).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.46 (2).jpeg
  Apagado com sucesso.
- /content/WhatsApp Image 2025-05-17 at 12.52.49 (1).jpeg
  Apagado com sucesso.
- /cont